In [ ]:
%%bash
pip -q install trl
pip -q install peft
pip -q install torch
pip -q install datasets
pip -q install transformers
pip -q install peft

In [21]:
import torch
from trl import SFTTrainer
from datasets import load_from_disk, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import pandas as pd

In [28]:
# df = pd.read_csv('TRAIN_DATA.csv')
# df

# data_files = {"train":"TRAIN_DATA.parquet","test":"TEST_DATA.parquet"}
# raw_datasets = load_dataset("parquet", data_dir='./', data_files=data_files)
# raw_datasets

# train_dataset = load_from_disk('./TRAIN_DATA.parquet')

# from datasets import load_dataset
dataset = load_dataset('csv', data_files='TRAIN_DATA.csv')
dataset_A = dataset['train']

# dataset = load_dataset('csv', data_files={'train': 'TRAIN_DATA.csv', 'test': 'TRAIN_DATA.csv'})
dataset_A

Dataset({
    features: ['input', 'instruction', 'output', 'text'],
    num_rows: 17500
})

In [4]:
import textwrap

In [5]:
for index in range(3):
    print("---"*15)
    print("Instruction: {}".format(textwrap.fill(df.iloc[index]["instruction"],width=50)))
    print("Output: {}".format(textwrap.fill(df.iloc[index]["output"],width=50)))
    print("Text: {}".format(textwrap.fill(df.iloc[index]["text"],width=50)))

---------------------------------------------
Instruction: Reduce stress. Find a way to manage the workload
without constant burnout.
Output: Delegate tasks to colleagues where possible.
Prioritize tasks based on urgency and importance.
Speak to a manager about workload concerns. Break
large tasks into smaller, manageable parts.
Implement time management techniques, such as the
Pomodoro technique.
Text: Delegating tasks may lighten the load but depends
on team dynamics. Prioritizing tasks will likely
be effective as it focuses on key deliverables.
Speaking to the manager might result in either
assistance or further stress, depending on the
relationship. Breaking tasks down will help
maintain productivity without feeling overwhelmed.
Time management techniques are easy to implement
and could help immediately.
---------------------------------------------
Instruction: Reduce the frequency of arguments. Improve
communication with the partner. Foster a more
positive and supportive relation

In [6]:
pretrained_model_name = "Salesforce/xgen-7b-8k-base"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name, torch_dtype=torch.bfloat16)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)

In [8]:
model_training_args = TrainingArguments(
    output_dir="xgen-7b-8k-base-fine-tuned",
    per_device_train_batch_size=4,
    optim="adamw_torch",
    logging_steps=80,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    num_train_epochs=100,
    save_strategy="epoch"
)

In [ ]:
lora_peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, task_type="CAUSAL_LM")

In [29]:
SFT_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_A,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=model_training_args,
    peft_config=lora_peft_config,
)

/Users/jan/DataspellProjects/ips_llama/venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/jan/DataspellProjects/ips_llama/venv/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/Users/jan/DataspellProjects/ips_llama/venv/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

RuntimeError: MPS backend out of memory (MPS allocated: 9.04 GB, other allocations: 384.00 KB, max allowed: 9.07 GB). Tried to allocate 86.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).